# Discord Question Generation

Original code: https://github.com/salesforce/discord_questions/blob/d7cbd514895bdfbb54782645909eea70fe1435b3/README.md

## Paper Version
**Incomplete**

Using their pipeline, which uses their pre-trained models to generate the discord questions, generate answers, and consolidate answers

the entire pipeline they used to create discord questions has more to it. Like removing duplicates, checking if the discord question actually create questions that made different sources answer differently (or, the sources can answer them in the first place), etc.

In [ ]:
# https://huggingface.co/Salesforce/discord_qg
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

qg_tokenizer = AutoTokenizer.from_pretrained("Salesforce/discord_qg")
qg_model = AutoModelForSeq2SeqLM.from_pretrained("Salesforce/discord_qg")

paragraph = "The International Monetary Fund warned on Tuesday that colliding pressures from inflation, war-driven energy and food crises and sharply higher interest rates were pushing the world to the brink of recession and threatening financial market stability."

for start_word in ["How", "Why"]:
    encoder_ids = qg_tokenizer.batch_encode_plus([paragraph], add_special_tokens=True, padding=True, truncation=True, return_tensors="pt")
    decoder_input_ids = qg_tokenizer.batch_encode_plus([start_word], add_special_tokens=True, return_tensors="pt")["input_ids"][:, :-1]
    model_output = qg_model.generate(**encoder_ids, decoder_input_ids=decoder_input_ids, max_length=20)
    generated_questions = qg_tokenizer.batch_decode(model_output, skip_special_tokens=True)

    print(generated_questions)


config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

['How will the global economy be affected by the crises?']
['Why is the world in recession?']


In [ ]:
# https://huggingface.co/Salesforce/discord_qa
# Load model directly
from transformers import AutoModelForQuestionAnswering

qa_tokenizer = AutoTokenizer.from_pretrained("Salesforce/discord_qa")
qa_model = AutoModelForQuestionAnswering.from_pretrained("Salesforce/discord_qa")


Consolidation process explanation
(note: the entire consolidation is in their github repo. below is just on using the language model they trained to calculate similarity)

they pair up all the possible pair-wise combinations of paragraphs. For every two paragraph (`p1` and `p2`) with answer fields (i assume here it's `a1`, `a2`), they create the text
`text = "<question> <sep> <p1['answer']> <sep> <p2['answer']>"`
which are then tokenized, padded, and passed through the model to obtain logits. The scores extracted from the logits are used to rank or measure the similarity of the two answers in the pair.

Once all pair-wise paragraph combination is calculated, they create a weight matrix, then use a threshold to determine if an edge should exist between two paragraphs, then use the weight matrix to create a graph. Finally run *Louvain algorithm* on the graph to determine clusters

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import itertools

ae_tokenizer = AutoTokenizer.from_pretrained("Salesforce/qa_consolidation")
ae_model = AutoModelForSequenceClassification.from_pretrained("Salesforce/qa_consolidation").eval()

question = "When will the recession happen?"
answers = ["probably next January", "never", "we're already in a recession", "it won't happen", "it's going on right now", "not before next year", "upcoming January-March"]
dataset = [{"a1": a1, "a2": a2, "input": "%s <sep> %s <sep> %s" % (question, a1, a2)} for a1, a2 in itertools.combinations(answers, 2)]

input_ids = ae_tokenizer.batch_encode_plus([d["input"] for d in dataset], add_special_tokens=False, padding=True, return_tensors="pt")["input_ids"]
scores = ae_model(input_ids=input_ids)["logits"][:, 0].tolist()
for d, score in zip(dataset, scores):
    d["score"] = score

for d in sorted(dataset, key=lambda d: -d["score"]):
    print("[Score: %.3f] %s" % (d["score"], d["input"]))


# Zero-Shot/Few-Shot Version

gonna try to recreate the Discord Question creation process just using base models